# PoC: Airlines Data Cleaning and Ingestion

In [1]:
import pandas as pd
import numpy as np

PROJ_PATH = "/data/"

DATA_IN_PREFIX = "orig/"
DATA_OUT_PREFIX = "processed/"


airlines_raw = pd.read_excel(PROJ_PATH + DATA_IN_PREFIX + "20180102 Airlines.xlsx")
airports_raw = pd.read_excel(PROJ_PATH + DATA_IN_PREFIX + "20180102 Airports.xlsx")
routes_raw = pd.read_excel(PROJ_PATH + DATA_IN_PREFIX + "20180102 Routes.xlsx")

## Data Exploration

In [2]:
airlines_raw.head()

## RES: First line looks dodgy. Remove?



## Look at airline types.
print(airlines_raw.dtypes)

#%%
# What percentages of actual missing data do we have?
# Note: Missing means NaN; there are other types of flaws in the data which we'll get to later.

def print_perc_missing(df):
    """
    Prints what percentage of data is missing for each column.
    
    CAVEAT: Only checks for nan, not other data defects, e.g. "\\N" entries etc.
    """
    print(round(df.isnull().sum() / df.shape[0] * 100))
    
print("\nAirlines data\n")
print_perc_missing(airlines_raw)

## RES:
## Airline aliases flawed.
## Most IATA codes are rubbish. Many nan, and other issues.
## ICAO codes have nans and other problems.
## Further, 13% of airline callsigns are flawed.


#%%
print("\nAirports data\n\n")
print_perc_missing(airports_raw)
## Airport cities seems to be an issue. 
## Timezone offset too.
## NOTE: Some IATA codes are \N --> not NaN! Need to fix this too.
## Need to fix cities if we want to use it as a key.
## There are two unnamed columns. Could probably just delete those as they only contain NaNs.

#%%
print("\nRoutes data\n\n")
print_perc_missing(routes_raw)

## RES: Codeshare is totally useless. Rest seems OK for now.

Dimensions...
(6162, 8)
(7184, 14)
(67663, 9)
Data types...
Airline ID               int64
Airline Name            object
Airline Alias           object
IATA Code               object
ICAO Code               object
Airline Callsign        object
Airline Country         object
Airline Operational?    object
dtype: object
Airport ID           int64
Airport Name        object
Airport City        object
Airport Country     object
IATA Code           object
ICAO Code           object
Latitude           float64
Longitude          float64
Altitude             int64
Timezone Offset     object
DST                 object
Timezone            object
Unnamed: 12        float64
Unnamed: 13        float64
dtype: object
Airline                   object
Airline ID                object
Source Airport            object
Source Airport ID         object
Destination Airport       object
Destination Airport ID    object
Codeshare?                object
No Stops                  object
Equipment             

In [ ]:
all_dta = [airlines_raw, airports_raw, routes_raw]

## Print dimensions.
print("Dimensions...")
list(map(lambda x: print(x.shape), all_dta))

## Print types.
print("Data types...")
list(map(lambda x: print(x.dtypes), all_dta))

## RES: routes has the largest amount of entries, ~68k rows.
## Probably also the most scalable part of the data as new routes
## will be created, or existing ones updated.
## Need to understand unit of scale in new routes.